In [13]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os

In [14]:
df = pd.read_csv('../data/ml_features_subset.csv')
df.head()

,date,spike_30,past_spikes_30,wind_speed,precipitation,hdd,cdd,past_da_load_error,past_da_price_error,month,hour,is_weekend,is_holiday,load_capacity_ratio
0,2018-01-01 01:00:00,0,2.0,7.127636,0.0,31.102500,0.0,1.527919e+06,10950.736579,1,1,0,0,1.083770
1,2018-01-01 02:00:00,0,2.0,8.209263,0.0,31.252500,0.0,1.514983e+06,10831.041829,1,2,0,0,0.857405
2,2018-01-01 03:00:00,0,2.0,9.832680,0.0,31.052501,0.0,1.513766e+06,10264.167962,1,3,0,0,0.824815
3,2018-01-01 04:00:00,0,2.0,9.028754,0.0,30.902500,0.0,1.521223e+06,10335.796294,1,4,0,0,0.825660
4,2018-01-01 05:00:00,0,2.0,9.085988,0.0,31.252500,0.0,1.535811e+06,10329.139457,1,5,0,0,0.814085


In [ ]:
df.drop(columns= 'date', inplace= True)

spike_30               6.330716e-02
past_spikes_30         1.520120e+00
wind_speed             1.232964e+01
precipitation          1.506831e-01
hdd                    7.692262e+00
cdd                    1.649448e+00
past_da_load_error     1.297536e+06
past_da_price_error    3.738954e+04
month                  6.547559e+00
hour                   1.150240e+01
is_weekend             2.832571e-01
is_holiday             1.323094e-03
load_capacity_ratio    6.324037e+01
dtype: float64

In [19]:
print(df.mean())
print(df.std())

spike_30               6.330716e-02
past_spikes_30         1.520120e+00
wind_speed             1.232964e+01
precipitation          1.506831e-01
hdd                    7.692262e+00
cdd                    1.649448e+00
past_da_load_error     1.297536e+06
past_da_price_error    3.738954e+04
month                  6.547559e+00
hour                   1.150240e+01
is_weekend             2.832571e-01
is_holiday             1.323094e-03
load_capacity_ratio    6.324037e+01
dtype: float64
spike_30               2.435181e-01
past_spikes_30         2.423469e+00
wind_speed             7.004305e+00
precipitation          1.408874e+00
hdd                    7.632753e+00
cdd                    3.050498e+00
past_da_load_error     2.008258e+06
past_da_price_error    2.321310e+05
month                  3.452787e+00
hour                   6.921556e+00
is_weekend             4.505867e-01
is_holiday             3.635081e-02
load_capacity_ratio    1.160576e+04
dtype: float64


In [16]:
plt.plot(df['price_error'])

KeyError: 'price_error'

In [ ]:
df.columns

Index(['date', 'hdd', 'cdd', 'temperature', 'wind_speed',
       'load_capacity_ratio', 'load_error', 'price_error', 'n_spikes_30',
       'n_spikes_45', 'n_spikes_60', 'past_da_load_error',
       'past_da_price_error'],
      dtype='object')